In [8]:
import argparse

import time
import random

import torch
import torch.nn as nn
from torch import optim
import numpy as np
import torch.nn.functional as F
from torch import Tensor
from nltk import bleu_score
from helper import Helper
from data import Data
from embedding_google import Get_Embedding
from train_network import Train_Network
from run_iterations import Run_Iterations
from manhattan_lstm import Manhattan_LSTM
import torch.nn.utils.rnn as rnn

In [9]:
use_cuda = torch.cuda.is_available()

In [10]:
data_name = "quora"
data_file = "../Datasets/quora.tsv"
embd_file = "../Embeddings/GoogleNews-vectors-negative300.bin.gz"
training_ratio = 0.8
max_len = 20
tracking_pair = False
hidden_size = 50
batch_size = 32
num_iters = 7
learning_rate = 0.001

data = Data(data_name, data_file, training_ratio, max_len)

print('\n')
print('Number of training samples        :', len(data.x_train))
print('Number of validation samples      :', len(data.x_val))
print('Maximum sequence length           :', max_len)
print('\n')

print('Building Embedding Matrix')
embedding = Get_Embedding(embd_file, data.word2index)
embedding_size = embedding.embedding_matrix.shape[1]

Number of Training Positive Samples   : 119341.0
Number of Training Negative Samples   : 204039.0
Number of Validation Positive Samples   : 29961.0
Number of Validation Negative Samples   : 50877.0


Number of training samples        : 323380
Number of validation samples      : 80838
Maximum sequence length           : 20


Building Embedding Matrix


In [11]:
print('Building model.')
model = Manhattan_LSTM(data_name, hidden_size, embedding.embedding_matrix, use_embedding=True, train_embedding=True)
if use_cuda: model = model.cuda()

model.init_weights()

Building model.


In [ ]:
print("Training Network.")
train_network = Train_Network(model, data.index2word)

run_iterations = Run_Iterations(data_name, train_network, data.x_train, data.y_train, data.index2word,
                                batch_size, num_iters, learning_rate,
                                tracking_pair=tracking_pair, x_val=data.x_val, y_val=data.y_val)
run_iterations.train_iters()
run_iterations.evaluate_randomly()

torch.save(model.state_dict(), './manhattan_lstm.pt')

Training Network.
Beginning Model Training.

4m 41s (- 28m 7s) (1 14%) 1916.5550
Validation Accuracy: 77.868082 Validation Precision: 74.540501 Validation Recall: 61.182871 Validation Loss: 385.882116


9m 42s (- 24m 17s) (2 28%) 1323.9802
Validation Accuracy: 78.925753 Validation Precision: 73.870646 Validation Recall: 66.750108 Validation Loss: 372.774588


14m 44s (- 19m 39s) (3 42%) 1071.1913
Validation Accuracy: 78.611544 Validation Precision: 72.462329 Validation Recall: 68.215347 Validation Loss: 376.400156


19m 47s (- 14m 50s) (4 57%) 902.8452
Validation Accuracy: 78.532373 Validation Precision: 72.166544 Validation Recall: 68.495711 Validation Loss: 382.227610


24m 49s (- 9m 55s) (5 71%) 789.5607
Validation Accuracy: 78.440832 Validation Precision: 71.724736 Validation Recall: 69.053102 Validation Loss: 387.421276


29m 50s (- 4m 58s) (6 85%) 689.6404
Validation Accuracy: 78.575670 Validation Precision: 71.923557 Validation Recall: 69.213311 Validation Loss: 387.387762


